# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. This notebook tests several attributes of the generation process to ensure that the code is sound. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
form = {
    "accountid": "3025284578519",
    "aggression": 0.5,
    "conformity": 0.5,
    "constitution": 0.5,
    "consumes": "organics",
    "effuses": "organic waste,plastics",
    "label": "form",
    "literacy": 0.5,
    "minerals": 0.5,
    "name": "worldgenform",
    "num_moons": 10,
    "num_planets": 4,
    "objid": 300882390249,
    "organics": 0.5,
    "owner": "William Harding",
    "starting_pop": 7,
    "userguid": "8d5b667f-b225-4641-b499-73b77558ff86",
    "username": "William Harding"
}

userguid = form["userguid"]

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(form)
system

<system: ordered; 3478695812083; Huage>

In [7]:
system.get_data()

{'objid': '3478695812083',
 'name': 'Huage',
 'label': 'system',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': 44.604,
 'glon': -10.665,
 'gelat': -3.15}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Par',
 'class': 'G',
 'objid': '2614382718954',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Par', 'class': 'G', 'objid': '2614382718954', 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Bhamoganj',
 'class': 'terrestrial',
 'objid': '5237515590885',
 'label': 'planet',
 'radius': 0.496,
 'mass': 0,
 'orbitsDistance': 1.108,
 'orbitsId': '2614382718954',
 'orbitsName': 'Par',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(form["num_planets"]) - 1)
]
planets

[<planet: dwarf; 8334033562884; Hymo>,
 <planet: dwarf; 9613914945085; Zihi>,
 <planet: terrestrial; 1001361712615; Otu>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(form["num_moons"]))
]
moons

[<moon: terrestrial; 0102295433743; Uwe>,
 <moon: rocky; 9054424676045; Mau>,
 <moon: ice; 9121662875625; Monmilherdo>,
 <moon: rocky; 3307258450819; Iscu>,
 <moon: rocky; 5819631318974; Walhor>,
 <moon: terrestrial; 5246684363916; Ciaharnew>,
 <moon: rocky; 5257102396718; Ferhradong>,
 <moon: rocky; 8853385327178; Jobonsel>,
 <moon: rocky; 6229017292868; Plavil>,
 <moon: terrestrial; 6196905957740; Vilamchanru>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: dwarf; 8334033562884; Hymo>

In [14]:
home_planet.orbiting

<star: G; 2614382718954; Par>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<system: ordered; 3478695812083; Huage>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,3478695812083,Huage,system,ordered,True,44.604,-10.665,-3.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2614382718954,Par,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,0102295433743,Uwe,moon,terrestrial,NaN,NaN,NaN,NaN,0.010545,8334033562884,Hymo,0.1281,0.005550,False,False
3,9054424676045,Mau,moon,rocky,NaN,NaN,NaN,NaN,0.002499,8334033562884,Hymo,0.2080,0.000568,False,False
4,9121662875625,Monmilherdo,moon,ice,NaN,NaN,NaN,NaN,0.005072,8334033562884,Hymo,0.1610,0.008329,False,False
5,3307258450819,Iscu,moon,rocky,NaN,NaN,NaN,NaN,0.103548,1001361712615,Otu,0.3810,0.000480,False,False
6,5819631318974,Walhor,moon,rocky,NaN,NaN,NaN,NaN,0.064227,1001361712615,Otu,0.3611,0.000566,False,False
7,5246684363916,Ciaharnew,moon,terrestrial,NaN,NaN,NaN,NaN,0.038673,8334033562884,Hymo,0.2010,0.008815,False,False
8,5257102396718,Ferhradong,moon,rocky,NaN,NaN,NaN,NaN,0.080800,1001361712615,Otu,0.3611,0.000467,False,False
9,8853385327178,Jobonsel,moon,rocky,NaN,NaN,NaN,NaN,0.008812,8334033562884,Hymo,0.1281,0.000439,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,5237515590885,2614382718954,orbits,1.1080
1,8334033562884,2614382718954,orbits,39.1610
2,9613914945085,2614382718954,orbits,40.4240
3,1001361712615,2614382718954,orbits,1.1210
4,0102295433743,8334033562884,orbits,0.0001
5,9054424676045,8334033562884,orbits,0.0800
6,9121662875625,8334033562884,orbits,0.0330
7,3307258450819,1001361712615,orbits,0.0200
8,5819631318974,1001361712615,orbits,0.0001
9,5246684363916,8334033562884,orbits,0.0730


In [18]:
pd.DataFrame(orbiting_edges).groupby(["node1", "node2"]).count()["label"].mean() == 1

True

In [19]:
pd.DataFrame(orbiting_edges).groupby(["node2"]).count()

,node1,label,orbit_distance
node2,,,
1001361712615,3,3,3
2614382718954,4,4,4
8334033562884,6,6,6
9613914945085,1,1,1


Check the orbiting logic

In [20]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'dwarf planet orbits a G star',
 'dwarf planet orbits a G star',
 'terrestrial planet orbits a G star',
 'terrestrial moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'ice moon orbits a dwarf planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'terrestrial moon orbits a dwarf planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a dwarf planet',
 'rocky moon orbits a dwarf planet',
 'terrestrial moon orbits a dwarf planet']

In [21]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,5237515590885,3478695812083,isIn
1,8334033562884,3478695812083,isIn
2,9613914945085,3478695812083,isIn
3,1001361712615,3478695812083,isIn
4,0102295433743,3478695812083,isIn
5,9054424676045,3478695812083,isIn
6,9121662875625,3478695812083,isIn
7,3307258450819,3478695812083,isIn
8,5819631318974,3478695812083,isIn
9,5246684363916,3478695812083,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [22]:
home_planet.scan_body()
home_planet.atmosphere

[{'Argon': 0.246},
 {'Carbon Dioxide': 0.223},
 {'Helium': 0.082},
 {'Hydrogen': 0.193},
 {'Methane': 0.006},
 {'Nitrogen': 0.054},
 {'Oxygen': 0.109},
 {'Sodium': 0.087}]

In [24]:
home_planet.get_data()

{'name': 'Bhamoganj',
 'class': 'terrestrial',
 'objid': '5237515590885',
 'label': 'planet',
 'atmosphere': [{'Argon': 0.246},
  {'Carbon Dioxide': 0.223},
  {'Helium': 0.082},
  {'Hydrogen': 0.193},
  {'Methane': 0.006},
  {'Nitrogen': 0.054},
  {'Oxygen': 0.109},
  {'Sodium': 0.087}],
 'radius': 0.496,
 'mass': 0,
 'orbitsDistance': 1.108,
 'orbitsId': '2614382718954',
 'orbitsName': 'Par',
 'isSupportsLife': False,
 'isPopulated': False}

In [25]:
home_planet.resources

[<resource: 4922300816520; organics>,
 <resource: 7687415333659; common minerals>,
 <resource: 3694306733951; rare minerals>,
 <resource: 9210630543932; water>]

In [26]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,organics,4922300816520,resource,976,976,bilogical material that can be consumed by pops,10.0
1,common minerals,7687415333659,resource,115,115,Iron and other common material used in constru...,NaN
2,rare minerals,3694306733951,resource,50,50,"lithium, silver and other rare minerals used i...",NaN
3,water,9210630543932,resource,10160,10160,"H2O ready to be consumed, either frozen or in ...",NaN


In [27]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,5237515590885,4922300816520,has
1,5237515590885,7687415333659,has
2,5237515590885,3694306733951,has
3,5237515590885,9210630543932,has


In [28]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 976 organics',
 'terrestrial planet has 115 common minerals',
 'terrestrial planet has 50 rare minerals',
 'terrestrial planet has 10160 water']